In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# Path Setup

import os

CodeDir = os.getcwd()
BaseDir = os.path.split(CodeDir)[0]
DataDir = os.path.join(os.path.split(CodeDir)[0], 'Data')

print("Code Directory:",CodeDir)
print("Data Directory:",DataDir)

print("Contents of the Data Directory:",os.listdir(DataDir))
print("Contents of the Data Directory:",os.listdir(CodeDir))

Code Directory: /home/slayer/Git/VindiataInternshipTest/Code
Data Directory: /home/slayer/Git/VindiataInternshipTest/Data
Contents of the Data Directory: ['AircraftType.csv', 'CityPairs.csv', 'Operations.csv']
Contents of the Data Directory: ['Untitled.ipynb', 'DataManipulation.ipynb', '.ipynb_checkpoints']


In [3]:
def csv_files(s):
    return '.csv' in s

csv_files = list(filter(csv_files,os.listdir(DataDir)))

In [4]:
AircraftType = pd.read_csv(os.path.join(DataDir,csv_files[0]))
CityPairs = pd.read_csv(os.path.join(DataDir,csv_files[1]))
Operations = pd.read_csv(os.path.join(DataDir,csv_files[2]))

In [5]:
AircraftType.head()

,Aircraft Type,Range (Km),Ave. Speed (km/h),Number of Seats,Costs per flight hour
0,A320,"5,000",800,150,"5,000"
1,A330,"8,000",900,250,"7,500"
2,B737,"5,000",800,150,"5,100"
3,B747,"10,000",900,350,"12,000"
4,Q400,"1,500",750,90,"3,500"


In [6]:
CityPairs.head()

,Origin City,Desitnation City,Pass. Demand \n(per day),Distance (km)
0,AA,BB,420,"3,000"
1,BB,CC,450,"6,500"
2,CC,AA,300,400
3,AA,DD,300,"1,000"


In [7]:
Operations.head()

,Aircraft Name,Aircraft Type,Jan-14,Feb-14,Mar-14,Apr-14,May-14,Jun-14,Jul-14,Aug-14,Sep-14,Oct-14,Nov-14,Dec-14
0,A,A320,250,251,248,253,243,245,248,246,256,256,251,252
1,B,A330,300,301,298,303,293,295,298,296,306,306,301,302
2,C,B737,245,246,243,248,238,240,243,241,251,251,246,247
3,D,B747,310,311,308,313,303,305,308,306,316,316,311,312
4,E,Q400,200,201,198,203,193,195,198,196,206,206,201,202


In [8]:
column_list = list(Operations)
Operations["SumOfHours"] = Operations[column_list[2:]].sum(axis = 1)

In [9]:
AircraftType['Costs per flight hour'] = pd.to_numeric(AircraftType['Costs per flight hour'].transform(lambda x:str(x).strip().replace(',','')))

In [10]:
CityPairs['Distance (km)'] = pd.to_numeric(CityPairs['Distance (km)'].transform(lambda x: str(x).strip().replace(',','')))

CityPairs.head()

,Origin City,Desitnation City,Pass. Demand \n(per day),Distance (km)
0,AA,BB,420,3000
1,BB,CC,450,6500
2,CC,AA,300,400
3,AA,DD,300,1000


In [11]:
AircraftType['Range (Km)'] = pd.to_numeric(AircraftType['Range (Km)'].transform(lambda x: str(x).strip().replace(',','')))

In [12]:
AircraftType.loc[AircraftType['Range (Km)'] >= CityPairs['Distance (km)'][0]]

,Aircraft Type,Range (Km),Ave. Speed (km/h),Number of Seats,Costs per flight hour
0,A320,5000,800,150,5000
1,A330,8000,900,250,7500
2,B737,5000,800,150,5100
3,B747,10000,900,350,12000


In [13]:
list(AircraftType)

['Aircraft Type',
 'Range (Km)',
 'Ave. Speed (km/h)',
 'Number of Seats',
 'Costs per flight hour']

In [14]:
list(CityPairs)

['Origin City',
 'Desitnation City',
 'Pass. Demand \n(per day)',
 'Distance (km)']

In [15]:
from math import ceil
def minCost(dist, n):
    t = AircraftType.loc[AircraftType['Range (Km)'] >= dist]
    t['cost per trip'] = (dist/t['Ave. Speed (km/h)'])*t['Costs per flight hour']
    t['no trips'] = n/t['Number of Seats'].apply(np.ceil).astype(int)
    t['Total Cost'] = t['cost per trip']*t['no trips']
    #print(t)
    m = min(t['Total Cost'])
    r = t.loc[t['Total Cost'] == m , 'Aircraft Type']
    del m,t
    return r.values[0]
    
    

In [16]:
CityPairs['AircraftName'] = np.vectorize(minCost)(CityPairs['Distance (km)'], CityPairs['Pass. Demand \n(per day)'])
CityPairs

/home/slayer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/slayer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/slayer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,Origin City,Desitnation City,Pass. Demand \n(per day),Distance (km),AircraftName
0,AA,BB,420,3000,A330
1,BB,CC,450,6500,A330
2,CC,AA,300,400,A330
3,AA,DD,300,1000,A330
